# many_model_absa_insert.ipynb 是 many_model_absa.ipynb(默认不加都是concat方式) 
# 的基础上，进行的insertion策略的实验结果记录

https://github.com/albert-jin/DictionaryFused-E2E-ABSA 仓库中的所有模型的训练记事本

## 包含了对lstm tdlstm tclstm ataelstm  ian memnet cabasc
## 对 acl2014data + semeval 2014,+15,+16 + aclshortdata 的 ABSA 模型表现实验

In [ ]:
!/opt/bin/nvidia-smi

Fri Sep  2 03:17:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

所有模型 include lstm,td_lstm,tc_lstm,atae_lstm,ian,memnet,ram,cabasc,tnet_lf,aoa,mgan,bert_spc,aen_bert,lcf_bert

In [ ]:
# 下载实验项目
!git clone https://github.com/albert-jin/DictionaryFused-E2E-ABSA.git

Cloning into 'DictionaryFused-E2E-ABSA'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 183 (delta 97), reused 148 (delta 74), pack-reused 0
Receiving objects: 100% (183/183), 2.07 MiB | 10.12 MiB/s, done.
Resolving deltas: 100% (97/97), done.


安装python解释器环境

In [ ]:
!pip install -r /content/DictionaryFused-E2E-ABSA/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 776.7 MB 4.5 kB/s 
     |████████████████████████████████| 1.3 MB 48.8 MB/s 
     |████████████████████████████████| 2.9 MB 46.3 MB/s 
     |████████████████████████████████| 1.1 MB 69.3 MB/s 
     |████████████████████████████████| 880 kB 70.8 MB/s 
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=f79dc1d4756cfc1d97d6ac026ecf31277e14968b488b7d01e17359b6560bbb94
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=a1739efb4fa1c7dfc7ec9b72e558f7adb271dcbdc92b071cdf680d8ef34fe27d
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sklearn sacremoses
  Attempting uninstall: torch
    Found exist

In [ ]:
#!pip install torch==1.7 # 需要兼容transformer包

     |████████████████████████████████| 776.7 MB 3.9 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.7.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.7.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.7.0 which is incompatible.


In [ ]:
# train.py embed_size == 200 则需要下载
!wget https://huggingface.co/stanfordnlp/glove/resolve/main/glove.twitter.27B.zip #twitter数据集,不区分大小写
!unzip glove.twitter.27B.zip -d /content/DictionaryFused-E2E-ABSA/glove_embeddings

--2022-09-02 03:22:42--  https://huggingface.co/stanfordnlp/glove/resolve/main/glove.twitter.27B.zip
Resolving huggingface.co (huggingface.co)... 52.202.207.64, 52.6.16.131, 2600:1f18:147f:e850:db35:e0c7:187b:c770, ...
Connecting to huggingface.co (huggingface.co)|52.202.207.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/stanfordnlp/glove/3123e7f5c3f6a30095d413b12fc3284bbf717acd2a9bed63d1c7c13bf5223352?response-content-disposition=attachment%3B%20filename%3D%22glove.twitter.27B.zip%22 [following]
--2022-09-02 03:22:42--  https://cdn-lfs.huggingface.co/stanfordnlp/glove/3123e7f5c3f6a30095d413b12fc3284bbf717acd2a9bed63d1c7c13bf5223352?response-content-disposition=attachment%3B%20filename%3D%22glove.twitter.27B.zip%22
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 65.9.86.27, 65.9.86.70, 65.9.86.14, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|65.9.86.27|:443... connected.
HTTP request sen

In [ ]:
!head -n 5 /content/DictionaryFused-E2E-ABSA/glove_embeddings/glove.twitter.27B.200d.txt

<user> 0.31553 0.53765 0.10177 0.032553 0.003798 0.015364 -0.20344 0.33294 -0.20886 0.10061 0.30976 0.50015 0.32018 0.13537 0.0087039 0.1911 0.24668 -0.060752 -0.43623 0.019302 0.59972 0.13444 0.012801 -0.54052 0.27387 -1.182 -0.27677 0.11279 0.46596 -0.090685 0.24253 0.15654 -0.23618 0.57694 0.17563 -0.01969 0.018295 0.37569 -0.41984 0.22613 -0.20438 -0.076249 0.40356 0.61582 -0.10064 0.23318 0.22808 0.34576 -0.14627 -0.1988 0.033232 -0.84885 -0.25684 0.26369 0.29562 0.1847 -0.20668 -0.013297 0.12233 -0.47751 -0.17202 -0.14577 0.047446 -0.15824 0.054215 -0.19426 -0.081484 0.099009 0.10159 0.043571 0.50245 0.13362 0.065985 0.032969 -0.2017 -0.56905 -0.13203 0.073347 -0.063728 -0.2796 -0.38481 -0.020193 0.22298 -0.059115 0.045198 -0.13995 -0.13299 0.47309 -0.021874 0.38758 -0.074926 -0.0028093 -0.29829 -0.074987 -0.58542 -0.18065 -0.041805 0.41938 0.41004 -0.5911 0.10459 0.10724 0.69768 -0.15901 -0.059596 0.29368 -0.19609 0.39124 -0.29333 -0.0050833 -0.37854 0.33858 0.24782 0.29144 -0.2

# 测试Glove模型在Twitter数据集上的表现

增加字典知识后：Training **twitter** dataset on model**(LSTM)**

In [ ]:
!pwd
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name lstm --dataset twitter_know --embed_dim 200 --patience 50 --log_step 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    glove_embeddings  many_model_absa.ipynb  requirements.txt
data_utils.py	    infer_example.py  models		     train_insert.py
deberta_abas.ipynb  layers	      README.md		     train.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.twitter.27B.200d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 1664.
> testing dataset count: 419.
cuda memory allocated: 17950720
> n_trainable_params: 603303, n_nontrainable_params: 3884000
> training arguments:
>>> model_name: lstm
>>> dataset: twitter_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f7adbfc3c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 30
>>> embed_dim: 200
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 50
>>> device: cuda
>>> seed: 1234
>>> valset_ratio: 

增加字典知识后：Training **twitter** dataset on model**(td_LSTM)**

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name lstm --dataset twitter_know --embed_dim 200 --patience 50 --log_step 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.twitter.27B.200d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 1664.
> testing dataset count: 419.
cuda memory allocated: 17950720
> n_trainable_params: 603303, n_nontrainable_params: 3884000
> training arguments:
>>> model_name: lstm
>>> dataset: twitter_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7fe37109cc20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 30
>>> embed_dim: 200
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 50
>>> device: cuda
>>> seed: 1234
>>> val

增加字典知识后：Training **twitter** dataset on model**(tc_LSTM)**

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name tc_lstm --dataset twitter_know --embed_dim 200 --patience 50 --log_step 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.twitter.27B.200d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 1245.
> testing dataset count: 332.
cuda memory allocated: 22284288
> n_trainable_params: 1686603, n_nontrainable_params: 3884000
> training arguments:
>>> model_name: tc_lstm
>>> dataset: twitter_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f452e2eac20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 30
>>> embed_dim: 200
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 50
>>> device: cuda
>>> seed: 1234
>>>

增加字典知识后: Training **twitter** dataset on model(**atae_lstm**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name atae_lstm --dataset twitter_know --embed_dim 200 --patience 50 --log_step 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.twitter.27B.200d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 1664.
> testing dataset count: 419.
cuda memory allocated: 22920704
> n_trainable_params: 1845303, n_nontrainable_params: 3884000
> training arguments:
>>> model_name: atae_lstm
>>> dataset: twitter_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7ff04eb68c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 30
>>> embed_dim: 200
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 50
>>> device: cuda
>>> seed: 1234
>

增加字典知识后: Training **Twitter** dataset on model(**IAN**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name ian --dataset twitter_know --embed_dim 200 --patience 50 --log_step 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.twitter.27B.200d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 1664.
> testing dataset count: 419.
cuda memory allocated: 23257088
> n_trainable_params: 1928403, n_nontrainable_params: 3884000
> training arguments:
>>> model_name: ian
>>> dataset: twitter_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f41d36e1c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 30
>>> embed_dim: 200
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 50
>>> device: cuda
>>> seed: 1234
>>> val

增加字典知识后: Training **Twitter** dataset on model(**memnet**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name memnet --dataset twitter_know --embed_dim 200 --patience 50 --log_step 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.twitter.27B.200d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 1664.
> testing dataset count: 419.
cuda memory allocated: 16186368
> n_trainable_params: 161803, n_nontrainable_params: 3884000
> training arguments:
>>> model_name: memnet
>>> dataset: twitter_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7fdbb9449c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 30
>>> embed_dim: 200
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 50
>>> device: cuda
>>> seed: 1234
>>> v

增加字典知识后: Training **Twitter** dataset on model(**cabasc**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name cabasc --dataset twitter_know --embed_dim 200 --patience 50 --log_step 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.twitter.27B.200d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 1245.
> testing dataset count: 332.
cuda memory allocated: 19801600
> n_trainable_params: 1065405, n_nontrainable_params: 3884000
> training arguments:
>>> model_name: cabasc
>>> dataset: twitter_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f05b8dcec20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 30
>>> embed_dim: 200
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 50
>>> device: cuda
>>> seed: 1234
>>> 

# 针对SemEval201X以及acl2014data的notebook上实验

lstm
tdlstm  
tclstm  
ataelstm  
ian 
memnet  
cabasc 

In [ ]:
# train.py embed_size == 300 则需要下载
!wget https://huggingface.co/stanfordnlp/glove/resolve/main/glove.42B.300d.zip #Common Crawl数据集,不区分大小写
!unzip glove.42B.300d.zip -d /content/DictionaryFused-E2E-ABSA/glove_embeddings

--2022-09-02 03:47:48--  https://huggingface.co/stanfordnlp/glove/resolve/main/glove.42B.300d.zip
Resolving huggingface.co (huggingface.co)... 52.202.207.64, 52.6.16.131, 2600:1f18:147f:e850:db35:e0c7:187b:c770, ...
Connecting to huggingface.co (huggingface.co)|52.202.207.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/stanfordnlp/glove/357baac33090f645e71e253b3295ee1b767c98a0336e9a1d99c77e9e33b43c4a?response-content-disposition=attachment%3B%20filename%3D%22glove.42B.300d.zip%22 [following]
--2022-09-02 03:47:48--  https://cdn-lfs.huggingface.co/stanfordnlp/glove/357baac33090f645e71e253b3295ee1b767c98a0336e9a1d99c77e9e33b43c4a?response-content-disposition=attachment%3B%20filename%3D%22glove.42B.300d.zip%22
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 65.9.86.70, 65.9.86.14, 65.9.86.11, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|65.9.86.70|:443... connected.
HTTP request sent, awaiti

增加字典知识后：Training **SemEval2014** dataset on model(**LSTM**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name lstm --dataset SemEval2014_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 3093.
> testing dataset count: 329.
cuda memory allocated: 18574848
> n_trainable_params: 723303, n_nontrainable_params: 3920100
> training arguments:
>>> model_name: lstm
>>> dataset: SemEval2014_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f8aa75e6c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> valset

# 增加字典知识后：Training **SemEval2014** dataset on model(**TDLSTM**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name td_lstm --dataset SemEval2014_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 2913.
> testing dataset count: 311.
cuda memory allocated: 21468160
> n_trainable_params: 1446603, n_nontrainable_params: 3920100
> training arguments:
>>> model_name: td_lstm
>>> dataset: SemEval2014_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f70bd039c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> va

# 增加字典知识后：Training **SemEval2014** dataset on model(**TCLSTM**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name tc_lstm --dataset SemEval2014_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 2913.
> testing dataset count: 311.
cuda memory allocated: 24348672
> n_trainable_params: 2166603, n_nontrainable_params: 3920100
> training arguments:
>>> model_name: tc_lstm
>>> dataset: SemEval2014_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7fe355766c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> va

# 增加字典知识后：Training **SemEval2014** dataset on model(**ATAELSTM**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name atae_lstm --dataset SemEval2014_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 3093.
> testing dataset count: 329.
cuda memory allocated: 25786368
> n_trainable_params: 2525703, n_nontrainable_params: 3920100
> training arguments:
>>> model_name: atae_lstm
>>> dataset: SemEval2014_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f6c42cc0c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> 

# 增加字典知识后：Training **SemEval2014** dataset on model(**IAN**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name ian --dataset SemEval2014_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 3093.
> testing dataset count: 329.
cuda memory allocated: 24360960
> n_trainable_params: 2168403, n_nontrainable_params: 3920100
> training arguments:
>>> model_name: ian
>>> dataset: SemEval2014_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f7b5ab90c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> valset

# 增加字典知识后：Training **SemEval2014** dataset on model(**MEMNET**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name memnet --dataset SemEval2014_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 3093.
> testing dataset count: 329.
cuda memory allocated: 17135616
> n_trainable_params: 362703, n_nontrainable_params: 3920100
> training arguments:
>>> model_name: memnet
>>> dataset: SemEval2014_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f384f788c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> vals

# 增加字典知识后：Training **SemEval2014** dataset on model(**CABASC**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name cabasc --dataset SemEval2014_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 2913.
> testing dataset count: 311.
cuda memory allocated: 21485568
> n_trainable_params: 1446005, n_nontrainable_params: 3920100
> training arguments:
>>> model_name: cabasc
>>> dataset: SemEval2014_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f34677c3c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> val

# 针对SemEval201X以及acl2014data的notebook上实验

# SemEval2015 数据集

lstm

tdlstm  
tclstm  
ataelstm  
ian 
memnet  
cabasc 


# 增加字典知识后：Training **SemEval2015** dataset on model(**LSTM**)

In [ ]:
!pwd
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name lstm --dataset SemEval2015_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train_insert.py
data_utils.py	    layers		   README.md	     train.py
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   state_dict
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 750.
> testing dataset count: 82.
cuda memory allocated: 9015808
> n_trainable_params: 723303, n_nontrainable_params: 1530300
> training arguments:
>>> model_name: lstm
>>> dataset: SemEval2015_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7fc0320b3c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 12


# 增加字典知识后：Training **SemEval2015** dataset on model(**TDLSTM**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name td_lstm --dataset SemEval2015_know --embed_dim 300 --patience 30


/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 658.
> testing dataset count: 79.
cuda memory allocated: 11909120
> n_trainable_params: 1446603, n_nontrainable_params: 1530300
> training arguments:
>>> model_name: td_lstm
>>> dataset: SemEval2015_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f7ea8867c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> vals

# 增加字典知识后：Training **SemEval2015** dataset on model(**TCLSTM**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name tc_lstm --dataset SemEval2015_know --embed_dim 300 --patience 30


/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 658.
> testing dataset count: 79.
cuda memory allocated: 14789632
> n_trainable_params: 2166603, n_nontrainable_params: 1530300
> training arguments:
>>> model_name: tc_lstm
>>> dataset: SemEval2015_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7fd235725c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> vals

# 增加字典知识后：Training **SemEval2015** dataset on model(**ATAELSTM**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name atae_lstm --dataset SemEval2015_know --embed_dim 300 --patience 30


/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 750.
> testing dataset count: 82.
cuda memory allocated: 16227328
> n_trainable_params: 2525703, n_nontrainable_params: 1530300
> training arguments:
>>> model_name: atae_lstm
>>> dataset: SemEval2015_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7fd802857c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> va

# 增加字典知识后：Training **SemEval2015** dataset on model(**IAN**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name ian --dataset SemEval2015_know --embed_dim 300 --patience 30


/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 750.
> testing dataset count: 82.
cuda memory allocated: 14801920
> n_trainable_params: 2168403, n_nontrainable_params: 1530300
> training arguments:
>>> model_name: ian
>>> dataset: SemEval2015_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f7a8c092c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> valset_r

# 增加字典知识后：Training **SemEval2015** dataset on model(**MEMNET**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name memnet --dataset SemEval2015_know --embed_dim 300 --patience 30


/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 750.
> testing dataset count: 82.
cuda memory allocated: 7576576
> n_trainable_params: 362703, n_nontrainable_params: 1530300
> training arguments:
>>> model_name: memnet
>>> dataset: SemEval2015_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f9c1ffcac20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> valset_

# 增加字典知识后：Training **SemEval2015** dataset on model(**CABASC**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name cabasc --dataset SemEval2015_know --embed_dim 300 --patience 30


/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 658.
> testing dataset count: 79.
cuda memory allocated: 11910144
> n_trainable_params: 1446005, n_nontrainable_params: 1530300
> training arguments:
>>> model_name: cabasc
>>> dataset: SemEval2015_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f96f1f16c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> valse

# 增加字典知识后：Training **SemEval2016** dataset on model(**LSTM**)

lstm
tdlstm  
tclstm  
ataelstm  
ian 
memnet  
cabasc 

In [ ]:
!pwd
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name lstm --dataset SemEval2016_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train_insert.py
data_utils.py	    layers		   README.md	     train.py
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   state_dict
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 1112.
> testing dataset count: 118.
cuda memory allocated: 10786816
> n_trainable_params: 723303, n_nontrainable_params: 1973100
> training arguments:
>>> model_name: lstm
>>> dataset: SemEval2016_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f063fdb9c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed:

# 增加字典知识后：Training **SemEval2016** dataset on model(**TDLSTM**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name td_lstm --dataset SemEval2016_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 1006.
> testing dataset count: 103.
cuda memory allocated: 13680128
> n_trainable_params: 1446603, n_nontrainable_params: 1973100
> training arguments:
>>> model_name: td_lstm
>>> dataset: SemEval2016_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f50de808c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> va

# 增加字典知识后：Training **SemEval2016** dataset on model(**TCLSTM**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name tc_lstm --dataset SemEval2016_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 1006.
> testing dataset count: 103.
cuda memory allocated: 16659456
> n_trainable_params: 2166603, n_nontrainable_params: 1973100
> training arguments:
>>> model_name: tc_lstm
>>> dataset: SemEval2016_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f67a8bfcc20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> va

# 增加字典知识后：Training **SemEval2016** dataset on model(**ATAELSTM**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name atae_lstm --dataset SemEval2016_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 1112.
> testing dataset count: 118.
cuda memory allocated: 17998336
> n_trainable_params: 2525703, n_nontrainable_params: 1973100
> training arguments:
>>> model_name: atae_lstm
>>> dataset: SemEval2016_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f3bff427c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> 

# 增加字典知识后：Training **SemEval2016** dataset on model(**IAN**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name ian --dataset SemEval2016_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 1112.
> testing dataset count: 118.
cuda memory allocated: 16572928
> n_trainable_params: 2168403, n_nontrainable_params: 1973100
> training arguments:
>>> model_name: ian
>>> dataset: SemEval2016_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f30b96afc20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> valset

# 增加字典知识后：Training **SemEval2016** dataset on model(**MEMNET**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name memnet --dataset SemEval2016_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 1112.
> testing dataset count: 118.
cuda memory allocated: 9347584
> n_trainable_params: 362703, n_nontrainable_params: 1973100
> training arguments:
>>> model_name: memnet
>>> dataset: SemEval2016_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f676702ac20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> valse

# 增加字典知识后：Training **SemEval2016** dataset on model(**CABASC**)

In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name cabasc --dataset SemEval2016_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 1006.
> testing dataset count: 103.
cuda memory allocated: 13681152
> n_trainable_params: 1446005, n_nontrainable_params: 1973100
> training arguments:
>>> model_name: cabasc
>>> dataset: SemEval2016_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f080a3c0c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> val

lstm
tdlstm  
tclstm  
ataelstm  
ian 
memnet  
cabasc 

# 增加字典知识后：Training **acl14shortdata** dataset on model(**LSTM**)


In [ ]:
!pwd
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name lstm --dataset acl14shortdata_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train_insert.py
data_utils.py	    layers		   README.md	     train.py
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   state_dict
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 5623.
> testing dataset count: 625.
cuda memory allocated: 19671552
> n_trainable_params: 723303, n_nontrainable_params: 4136100
> training arguments:
>>> model_name: lstm
>>> dataset: acl14shortdata_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f2302b12c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> se

# 增加字典知识后：Training **acl14shortdata** dataset on model(**TDLSTM**)



In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name td_lstm --dataset acl14shortdata_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 4917.
> testing dataset count: 539.
cuda memory allocated: 22564864
> n_trainable_params: 1446603, n_nontrainable_params: 4136100
> training arguments:
>>> model_name: td_lstm
>>> dataset: acl14shortdata_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f6ada9e3c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>>

# 增加字典知识后：Training **acl14shortdata** dataset on model(**TCLSTM**)


In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name tc_lstm --dataset acl14shortdata_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 4917.
> testing dataset count: 539.
cuda memory allocated: 25445376
> n_trainable_params: 2166603, n_nontrainable_params: 4136100
> training arguments:
>>> model_name: tc_lstm
>>> dataset: acl14shortdata_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f379e250c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>>

# 增加字典知识后：Training **acl14shortdata** dataset on model(**ATAELSTM**)



In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name atae_lstm --dataset acl14shortdata_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 5623.
> testing dataset count: 625.
cuda memory allocated: 26883072
> n_trainable_params: 2525703, n_nontrainable_params: 4136100
> training arguments:
>>> model_name: atae_lstm
>>> dataset: acl14shortdata_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7ff68945cc20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>

# 增加字典知识后：Training **acl14shortdata** dataset on model(**IAN**)


In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name ian --dataset acl14shortdata_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 5623.
> testing dataset count: 625.
cuda memory allocated: 25457664
> n_trainable_params: 2168403, n_nontrainable_params: 4136100
> training arguments:
>>> model_name: ian
>>> dataset: acl14shortdata_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7fdf1548ac20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> val

# 增加字典知识后：Training **acl14shortdata** dataset on model(**MEMNET**)


In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name memnet --dataset acl14shortdata_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 5623.
> testing dataset count: 625.
cuda memory allocated: 18232320
> n_trainable_params: 362703, n_nontrainable_params: 4136100
> training arguments:
>>> model_name: memnet
>>> dataset: acl14shortdata_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f5938919c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> v

# 增加字典知识后：Training **acl14shortdata** dataset on model(**CABASC**)


In [ ]:
!pwd
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && pwd && ls && python3 train_insert.py --model_name cabasc --dataset acl14shortdata_know --embed_dim 300 --patience 30

/content
/content/DictionaryFused-E2E-ABSA
datasets	    infer_example.py	   __pycache__	     train.py
data_utils.py	    layers		   README.md
deberta_abas.ipynb  many_model_absa.ipynb  requirements.txt
glove_embeddings    models		   train_insert.py
>>> 使用设备:cuda 训练.
加载预训练向量...
>>> 使用 ./glove_embeddings/glove.42B.300d.txt 作为预训练单词的向量.
预训练向量加载完毕.
> training dataset count: 4917.
> testing dataset count: 539.
cuda memory allocated: 22565888
> n_trainable_params: 1446005, n_nontrainable_params: 4136100
> training arguments:
>>> model_name: cabasc
>>> dataset: acl14shortdata_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f2712413c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 30
>>> device: cuda
>>> seed: 1234
>>> 

# 测试五个数据集在Bert_spc上的效果
**BERT_BASED - MODEL**

跑Bert前最好检测一下是否cuda可用

In [ ]:
import torch
torch.cuda.is_available()

True

增加字典知识后： Training **Twitter** dataset on model**(bert_spc)**

In [ ]:
!cd /content/DictionaryFused-E2E-ABSA && python3 train_insert.py --model_name bert_spc --dataset twitter_know --log_step 20  # batch_size 是16，所以每300个样本测一下

>>> 使用设备:cuda 训练.
加载Bert...
Downloading: 100% 232k/232k [00:00<00:00, 677kB/s]
Downloading: 100% 570/570 [00:00<00:00, 606kB/s]
Downloading: 100% 440M/440M [00:06<00:00, 73.3MB/s]
Bert加载完毕.
> training dataset count: 1664.
> testing dataset count: 419.
cuda memory allocated: 439075328
> n_trainable_params: 109484547, n_nontrainable_params: 0
> training arguments:
>>> model_name: bert_spc
>>> dataset: twitter_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f809f435c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 20
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 20
>>> device: cuda
>>> seed: 1234
>>> valset_ratio: 0
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.bert_spc.BERT_SPC'>
>>> dataset_file: {'train': './datasets/twitter/output_k

增加字典知识后：Training **SemEval2014** dataset on model**(Bert_spc)**

In [ ]:
!cd /content/DictionaryFused-E2E-ABSA && python3 train_insert.py --model_name bert_spc --dataset SemEval2014_know --log_step 100

>>> 使用设备:cuda 训练.
加载Bert...
Bert加载完毕.
> training dataset count: 3093.
> testing dataset count: 329.
cuda memory allocated: 439075328
> n_trainable_params: 109484547, n_nontrainable_params: 0
> training arguments:
>>> model_name: bert_spc
>>> dataset: SemEval2014_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f1683550c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 20
>>> device: cuda
>>> seed: 1234
>>> valset_ratio: 0
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.bert_spc.BERT_SPC'>
>>> dataset_file: {'train': './datasets/laprest14/output_know_insert/train.tsv', 'test': './datasets/laprest14/output_know_insert/dev.tsv'}
>>> inputs_cols: ['concat_bert_indices', 'concat_segments_indic

增加字典知识后：Training **SemEval2015** dataset on model**(bert_spc)**

In [ ]:
!cd /content/DictionaryFused-E2E-ABSA && python3 train_insert.py --model_name bert_spc --dataset SemEval2015_know --log_step 100

>>> 使用设备:cuda 训练.
加载Bert...
Downloading: 100% 232k/232k [00:00<00:00, 2.67MB/s]
Downloading: 100% 570/570 [00:00<00:00, 617kB/s]
Downloading: 100% 440M/440M [00:05<00:00, 75.8MB/s]
Bert加载完毕.
> training dataset count: 750.
> testing dataset count: 82.
cuda memory allocated: 439075328
> n_trainable_params: 109484547, n_nontrainable_params: 0
> training arguments:
>>> model_name: bert_spc
>>> dataset: SemEval2015_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f770c5dfc20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 20
>>> device: cuda
>>> seed: 1234
>>> valset_ratio: 0
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.bert_spc.BERT_SPC'>
>>> dataset_file: {'train': './datasets/rest15/outpu

增加字典知识后：Training **SemEval2016** dataset on model**(bert_spc)**

In [ ]:
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && python3 train_insert.py --model_name bert_spc --dataset SemEval2016_know --log_step 100

>>> 使用设备:cuda 训练.
加载Bert...
Bert加载完毕.
> training dataset count: 1112.
> testing dataset count: 118.
cuda memory allocated: 439075328
> n_trainable_params: 109484547, n_nontrainable_params: 0
> training arguments:
>>> model_name: bert_spc
>>> dataset: SemEval2016_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f13fbfe1c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 20
>>> device: cuda
>>> seed: 1234
>>> valset_ratio: 0
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.bert_spc.BERT_SPC'>
>>> dataset_file: {'train': './datasets/rest16/output_know_insert/train.tsv', 'test': './datasets/rest16/output_know_insert/dev.tsv'}
>>> inputs_cols: ['concat_bert_indices', 'concat_segments_indices']
>

增加字典知识后：Training **acl14shortdata** dataset on model**(bert_spc)**

In [ ]:
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && python3 train_insert.py --model_name bert_spc --dataset acl14shortdata_know --log_step 100

>>> 使用设备:cuda 训练.
加载Bert...
Bert加载完毕.
> training dataset count: 5623.
> testing dataset count: 625.
cuda memory allocated: 439075328
> n_trainable_params: 109484547, n_nontrainable_params: 0
> training arguments:
>>> model_name: bert_spc
>>> dataset: acl14shortdata_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f493dd53c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 100
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 20
>>> device: cuda
>>> seed: 1234
>>> valset_ratio: 0
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.bert_spc.BERT_SPC'>
>>> dataset_file: {'train': './datasets/acl14shortdata/output_know_insert/train.tsv', 'test': './datasets/acl14shortdata/output_know_insert/dev.tsv'}
>>> inputs_cols: ['concat_bert_indices', 'concat_s

跑Bert前最好检测一下是否cuda可用

In [ ]:
import torch
torch.cuda.is_available()

增加字典知识后： Training **Twitter** dataset on model**(lfc_bert)**

In [ ]:
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && python3 train_insert.py --model_name lcf_bert --dataset twitter_know --log_step 20 --patience 5 --max_seq_len 125  # batch_size 是16，所以每300个样本测一下

>>> 使用设备:cuda 训练.
加载Bert...
Bert加载完毕.
> training dataset count: 1664.
> testing dataset count: 419.
cuda memory allocated: 455608832
> n_trainable_params: 113617923, n_nontrainable_params: 0
> training arguments:
>>> model_name: lcf_bert
>>> dataset: twitter_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f583dd45c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 20
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda
>>> seed: 1234
>>> valset_ratio: 0
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.lcf_bert.LCF_BERT'>
>>> dataset_file: {'train': './datasets/twitter/output_know_insert/train.tsv', 'test': './datasets/twitter/output_know_insert/dev.tsv'}
>>> inputs_cols: ['concat_bert_indices', 'concat_segments_indices', 'text

增加字典知识后： Training **SemEval2014** dataset on model**(lfc_bert)**

In [ ]:
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && python3 train_insert.py --model_name lcf_bert --dataset SemEval2014_know --log_step 20 --patience 5 --max_seq_len 125  # batch_size 是16，所以每300个样本测一下

>>> 使用设备:cuda 训练.
加载Bert...
Bert加载完毕.
> training dataset count: 3093.
> testing dataset count: 329.
cuda memory allocated: 455608832
> n_trainable_params: 113617923, n_nontrainable_params: 0
> training arguments:
>>> model_name: lcf_bert
>>> dataset: SemEval2014_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f85ae5afc20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 20
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda
>>> seed: 1234
>>> valset_ratio: 0
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.lcf_bert.LCF_BERT'>
>>> dataset_file: {'train': './datasets/laprest14/output_know_insert/train.tsv', 'test': './datasets/laprest14/output_know_insert/dev.tsv'}
>>> inputs_cols: ['concat_bert_indices', 'concat_segments_indices

增加字典知识后： Training **SemEval2015** dataset on model**(lfc_bert)**

In [1]:
# 下载实验项目
!git clone https://github.com/albert-jin/DictionaryFused-E2E-ABSA.git
!pip install -r /content/DictionaryFused-E2E-ABSA/requirements.txt

Cloning into 'DictionaryFused-E2E-ABSA'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 183 (delta 97), reused 148 (delta 74), pack-reused 0
Receiving objects: 100% (183/183), 2.07 MiB | 5.21 MiB/s, done.
Resolving deltas: 100% (97/97), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 776.7 MB 4.5 kB/s 
     |████████████████████████████████| 1.3 MB 55.9 MB/s 
     |████████████████████████████████| 1.1 MB 60.8 MB/s 
     |████████████████████████████████| 2.9 MB 39.8 MB/s 
     |████████████████████████████████| 880 kB 66.0 MB/s 
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=35ea96357bcea034ac35b93f717860beff1538887a9b13fd3544534329b14a2d
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e

In [2]:
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && python3 train_insert.py --model_name lcf_bert --dataset SemEval2015_know --log_step 20 --patience 5 --max_seq_len 125  # batch_size 是16，所以每300个样本测一下

>>> 使用设备:cuda 训练.
加载Bert...
Downloading: 100% 232k/232k [00:00<00:00, 21.7MB/s]
Downloading: 100% 570/570 [00:00<00:00, 872kB/s]
Downloading: 100% 440M/440M [00:05<00:00, 74.4MB/s]
Bert加载完毕.
> training dataset count: 750.
> testing dataset count: 82.
cuda memory allocated: 455608832
> n_trainable_params: 113617923, n_nontrainable_params: 0
> training arguments:
>>> model_name: lcf_bert
>>> dataset: SemEval2015_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f82fed95c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 20
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda
>>> seed: 1234
>>> valset_ratio: 0
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.lcf_bert.LCF_BERT'>
>>> dataset_file: {'train': './datasets/rest15/output_

增加字典知识后： Training **SemEval2016** dataset on model**(lfc_bert)**

In [3]:
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict
!cd /content/DictionaryFused-E2E-ABSA && python3 train_insert.py --model_name lcf_bert --dataset SemEval2016_know --log_step 20 --patience 5 --max_seq_len 125  # batch_size 是16，所以每300个样本测一下

>>> 使用设备:cuda 训练.
加载Bert...
Bert加载完毕.
> training dataset count: 1112.
> testing dataset count: 118.
cuda memory allocated: 455608832
> n_trainable_params: 113617923, n_nontrainable_params: 0
> training arguments:
>>> model_name: lcf_bert
>>> dataset: SemEval2016_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7fb80e980c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 20
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda
>>> seed: 1234
>>> valset_ratio: 0
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.lcf_bert.LCF_BERT'>
>>> dataset_file: {'train': './datasets/rest16/output_know_insert/train.tsv', 'test': './datasets/rest16/output_know_insert/dev.tsv'}
>>> inputs_cols: ['concat_bert_indices', 'concat_segments_indices', 'te

增加字典知识后： Training **acl14shortdata** dataset on model**(lfc_bert)**

In [4]:
!rm -rf /content/DictionaryFused-E2E-ABSA/state_dict  
!cd /content/DictionaryFused-E2E-ABSA && python3 train_insert.py --model_name lcf_bert --dataset acl14shortdata_know --log_step 20 --patience 5 --max_seq_len 125  # batch_size 是16，所以每300个样本测一下

>>> 使用设备:cuda 训练.
加载Bert...
Bert加载完毕.
> training dataset count: 5623.
> testing dataset count: 625.
cuda memory allocated: 455608832
> n_trainable_params: 113617923, n_nontrainable_params: 0
> training arguments:
>>> model_name: lcf_bert
>>> dataset: acl14shortdata_know
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7ff6de210c20>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 100
>>> batch_size: 16
>>> log_step: 20
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 125
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda
>>> seed: 1234
>>> valset_ratio: 0
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.lcf_bert.LCF_BERT'>
>>> dataset_file: {'train': './datasets/acl14shortdata/output_know_insert/train.tsv', 'test': './datasets/acl14shortdata/output_know_insert/dev.tsv'}
>>> inputs_cols: ['concat_bert_indices', 'concat_seg

In [ ]:
# OVER